In [ ]:
!pip install srt
!pip install openai
!pip install langchain

In [ ]:
import subprocess
# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'video.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

In [ ]:
import os
import srt
import asyncio
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from srt import compose
import nest_asyncio 


nest_asyncio.apply() 

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = os.getenv('OPEN_AI_KEY')

# LLMの初期化
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

# 翻訳用のプロンプトテンプレート
template = """[translate to Japanese]:\n{text}"""
prompt = ChatPromptTemplate.from_template(template)

# ConversationBufferWindowMemoryの初期化
memory = ConversationBufferWindowMemory(k=5)  # 最後の5つの対話を記憶

# 翻訳チェインの定義
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

def read_srt(input_srt_file):
    """SRTファイルを読み込み、字幕データのリストを返す。

    Args:
        input_srt_file (str): 入力SRTファイルのパス

    Returns:
        list: 字幕データのリスト
    """
    try:
        with open(input_srt_file, 'r', encoding='utf-8', errors='ignore') as file:
            srt_data = file.read()
        return list(srt.parse(srt_data))
    except Exception as e:
        print(f"Error reading SRT file: {e}")
        return []

async def translate_subtitles(subtitles, batch_size=10):
    """字幕データをバッチ処理で翻訳する。

    Args:
        subtitles (list): 字幕データのリスト
        batch_size (int): バッチサイズ

    Returns:
        list: 翻訳された字幕データのリスト
    """
    translated_subtitles = []
    for i in range(0, len(subtitles), batch_size):
        batch = subtitles[i:i + batch_size]
        tasks = [chain.ainvoke({"text": subtitle.content}) for subtitle in batch]
        results = await asyncio.gather(*tasks)
        for j, result in enumerate(results):
            subtitles[i + j].content = result['text']
            progress = (i + j + 1) / len(subtitles) * 100
            print(f"Translation progress: {progress:.2f}%")
    return subtitles

def write_srt(output_srt_file, subtitles):
    """翻訳された字幕データをSRTファイルに書き込む。

    Args:
        output_srt_file (str): 出力SRTファイルのパス
        subtitles (list): 翻訳された字幕データのリスト
    """
    try:
        translated_srt_content = compose(subtitles)
        with open(output_srt_file, "w", encoding="utf-8") as f:
            f.write(translated_srt_content)
    except Exception as e:
        print(f"Error writing SRT file: {e}")

async def translate_srt(input_srt_file, output_srt_file, batch_size=10):
    """SRTファイルを読み込み、翻訳して、新しいSRTファイルに書き込む。

    Args:
        input_srt_file (str): 入力SRTファイルのパス
        output_srt_file (str): 出力SRTファイルのパス
        batch_size (int): バッチサイズ
    """
    subtitles = read_srt(input_srt_file)
    if not subtitles:
        print("No subtitles to translate.")
        return
    translated_subtitles = await translate_subtitles(subtitles, batch_size)
    write_srt(output_srt_file, translated_subtitles)

def add_line_breaks(text, max_length=20, max_lines=2):
    """テキストに改行を追加する。

    Args:
        text (str): 改行を追加するテキスト
        max_length (int): 1行の最大長
        max_lines (int): 最大行数

    Returns:
        str: 改行を追加したテキスト
    """
    result = ""
    current_line = ""
    lines = []

    for char in text:
        if char == "、" or char == "。":
            current_line += char
            if len(current_line) > max_length or char == "。":
                lines.append(current_line)
                current_line = ""
        else:
            current_line += char

    if current_line:
        lines.append(current_line)

    while len(lines) > max_lines:
        line1 = lines.pop(0)
        line2 = lines.pop(0)
        combined_line = line1 + line2
        lines.insert(0, combined_line)

    return "\n".join(lines)

def add_line_breaks_srt(srt_text):
    """SRT形式を保持して改行を追加する。

    Args:
        srt_text (str): SRT形式のテキスト

    Returns:
        str: 改行を追加したSRTテキスト
    """
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 実行
input_srt = 'sample.srt'
output_srt = 'translated_file.srt'

# SRT翻訳の実行
asyncio.run(translate_srt(input_srt, output_srt, batch_size=10))

# 改行されたテキストを取得
with open(output_srt, 'r', encoding='utf-8') as file:
    srt_text = file.read()

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)

# ファイルへの書き込み (エンコーディング指定)
with open("translated_file_result.srt", "w", encoding='utf-8') as file:
    file.write(result)


In [12]:
import subprocess

video = input_srt.replace(".srt", ".mp4")

subprocess.run([
    'ffmpeg',
    '-hwaccel', 'auto', 
    '-i', f"{video}",
    '-vf', "subtitles=translated_file_result.srt:force_style='FontName=Helvetica,FontSize=11'",
    '-c:v', 'h264_amf', 
    '-c:a', 'copy', 
    f'{video.replace(".mp4", "_jp.mp4")}' 
])

